In [10]:
import numpy as np
import pandas as pd
import random
import copy

In [11]:
matrix_km = pd.read_excel('df_distance_km.xlsx')
matrix_km.index = matrix_km.columns

class Vehicle():
    def __init__(self, id, capacidad_kg, costo_km, autonomia_km):
        self.id = id
        self.capacidad_kg = capacidad_kg
        self.costo_km = costo_km
        self.autonomia_km = autonomia_km
    def get_price(self, distance):
        return round(self.costo_km * distance, 2)

vehicles = []
for _, row in pd.read_excel('df_vehicle.xlsx').iterrows():
    vehicles.append(Vehicle(
        id = int(row['vehiculo_id']), 
        capacidad_kg= float(row['capacidad_kg']),
        costo_km= float(row['costo_km']), 
        autonomia_km= int(row['autonomia_km'])
        ))
vehiculos_2 = vehicles.copy()

class Client():
    def __init__(self, cliente, order_kg):
        self.cliente = cliente
        self.order_kg = order_kg

class Order():
    def __init__(self, clients, vehicle, route):
        self.clients = clients
        self.vehicle = vehicle
        self.route = route

    def get_clients(self):
        return self.clients
    
    def get_kg(self):
        return [i.order_kg for i in self.clients]
    
    def get_route(self):
        final_route = []
        for i in self.route:
            if i == 'Almacén':
                final_route.append([i, 0])
            for client in self.clients:
                if client.cliente == i:
                    final_route.append([i, client.order_kg])
        return final_route

In [12]:
def check_viability(ruta):
    for i in range(0,len(ruta)-1,1):
        try:
            if matrix_km[ruta[i].client][ruta[i+1].cliente] == 0:
                return False
        except:
            pass
    return True

def check_viability_client(ruta):
    for i in range(0,len(ruta)-1,1):
        try:
            if (matrix_km[ruta[i]][ruta[i+1]] != 0) == False:
                return False
        except:
            pass
    return True

def create_route(clients):
    random.shuffle(clients)
    route = clients
    while check_viability(route) == False:
        random.shuffle(clients)
        ['Almacén'] + clients + ['Almacén']
    return route

In [13]:
clientes = []
df_demandas = pd.read_excel('df_historic_order_demand.xlsx')
df_demandas[df_demandas['mes_anio'] == '12-2020']
for cliente, order_demand in zip(df_demandas[df_demandas['mes_anio'] == '12-2020']['cliente'], df_demandas[df_demandas['mes_anio'] == '12-2020']['order_demand']):
    clientes.append(Client(cliente, order_demand)) 


In [14]:
def generate_initial_solution(vehicles, route):
    solucion = {}
    clients = [i.cliente for i in route]
    random.shuffle(clients)

    vh_copy = vehicles.copy()
    random.shuffle(vh_copy)

    route_vehicles = vh_copy

    ruta = ['Almacén'] + [i.cliente for i in route] + ['Almacén']
    counter = 0
    while clients:
        bucle = True
        vehiculo = random.choice(route_vehicles)

        solucion[vehiculo.id] = {
            'clientes' : [],
            'distancia_total' : 0,
            'peso_total' : 0,
            'costo_total' : 0
        }

        #print('\n\nVehículo escogído: ',vehiculo.id, ' capacidad: ',vehiculo.capacidad_kg, ' autonomía: ', vehiculo.autonomia_km)
        carga_actual = 0
        distancia_actual = 0
        while clients and bucle:
            temp_route = ruta[counter:counter+2]
            distance = matrix_km[temp_route[0]][temp_route[1]]
            distance_to_warehouse = matrix_km[temp_route[0]]['Almacén']
            load = [i.order_kg for i in clientes if i.cliente == temp_route[1]][0]

            if (carga_actual+load)<=vehiculo.capacidad_kg and (distancia_actual+distance+distance_to_warehouse)<=vehiculo.autonomia_km:
                #print(clients)
                if distancia_actual == 0:
                    distancia_actual += (distance +distance_to_warehouse)
                else:
                    distancia_actual += distance
                #print(f'CARGA: {temp_route[1]} --> {[i.order_kg for i in clientes if i.cliente == temp_route[1]][0]}')
                carga_actual += load
                #print(f'Carga actual: {carga_actual}, Distancia actual: {distancia_actual}')
                #print (temp_route[0], ' --> ', temp_route[1],' ---> ', matrix_km[temp_route[0]][temp_route[1]],'km ',f'carga_actual: {carga_actual}', f'distancia_actual: {distancia_actual} \n')
                del clients[0]
                solucion[vehiculo.id]['clientes'].append(temp_route[1])
                counter += 1
            else:
                solucion[vehiculo.id]['distancia_total'] += round((distancia_actual+distance_to_warehouse),2)
                solucion[vehiculo.id]['peso_total'] += carga_actual
                solucion[vehiculo.id]['costo_total'] += round(vehiculo.costo_km * (distancia_actual+distance_to_warehouse),2)
                
                route_vehicles.remove(vehiculo)
                #print('\n ¡Has excedido la carga! \n ')
                bucle = False
        if len(clients) < 1:
            ultima_parada = ruta[-2]
            distancia_regreso = matrix_km[ultima_parada]['Almacén']
            solucion[vehiculo.id]['distancia_total'] += round(distancia_actual,2)
            solucion[vehiculo.id]['peso_total'] += carga_actual
            solucion[vehiculo.id]['costo_total'] += round(vehiculo.costo_km * distancia_actual,2)
            if distancia_regreso<=vehiculo.autonomia_km:
                distancia_actual += distancia_regreso
                #print(f'\nRegresando al Almacén desde {ultima_parada} ---> {distancia_regreso} km')
            else:
                route_vehicles.remove(vehiculo)
                #print('\n ¡Has excedido la carga! \n ')
    if check_viability_client([cliente for sublista in [k['clientes'] for i,k in solucion.items()] for cliente in sublista]) : 
        return(solucion)
    else:
        return 9999


route = generate_initial_solution(vehicles=vehicles, route=create_route(clientes))

In [15]:
def fitness(solution):
    if solution == 9999:
        return -9999
    else:
        total_cost = 0
        for vehicle_id, details in solution.items():
            total_cost += details['costo_total']  # Sumar el costo total de cada vehículo
        return -total_cost

def create_population(clients, size):
    population = []
    for _ in range(size):
        route = create_route(clients)
        population.append(route)
    return population

population = create_population(clientes, 100)
results = [fitness(generate_initial_solution(vehicles, i)) for i in population ]


In [16]:
def selection(population, results, tournament_size=5):
    tournament = random.sample(list(zip(population, results)), tournament_size)
    tournament.sort(key=lambda x: x[1], reverse=True)
    return [i[0] for i in tournament[0:2]]

def crossover(parents):
    # Une la mitad del padre 1 y la otra mitad del padre 2 mientras el cliente no esté en padre1
    parent1, parent2 = parents
    midpoint = len(parent1) // 2
    child = parent1[:midpoint] + [client for client in parent2 if client not in parent1[:midpoint]]
    
    if check_viability(child):
        return child
    else:
        return parent1


def mutate(route, mutation_rate=0.1):
    route_inicial = route.copy()
    # Coge una ruta y si entra dentro del 
    if  random.random() < mutation_rate:
        i, j = random.sample(range(0,len(route)), 2)
        route[i], route[j] = route[j], route[i]  # Intercambiar dos clientes
    if check_viability(route):
        return route
    else:
        return route_inicial

best_routes = selection(population, results, tournament_size=5)
new_route = mutate(crossover(best_routes))
len(new_route)



20

In [17]:
def genetic_algorithm(clients, vehicles, matrix_km, population_size=100, generations=200, mutation_rate=0.2):
    # Generar población inicial con sus resultados
    population = create_population(clients, population_size)
    results = [fitness(generate_initial_solution(vehicles, i)) for i in population ]  # Calcular fitness inicial
    routes = []

    for generation in range(generations):
        new_population = []
        new_results = []
        new_routes = []
        
        for _ in range(population_size):
            parents = selection(population, results)
            child = crossover(parents)
            child = mutate(child, mutation_rate)
            new_population.append(child)
        
        for j in new_population:
            route = generate_initial_solution(vehicles, j)
            route_value = fitness(route)

            new_results.append(route_value)
            new_routes.append(route)
        
        population = new_population
        results = new_results
        routes = new_routes

        # Mejor solución en la generación actual
        best_cost = min([i for i in results if i != -9999])
        print(f"Generación {generation}: Mejor costo: {-best_cost}")  # Invertir para mostrar el costo original
    
    # Mejor solución final
    best_index = results.index(max(results))  # Seleccionar el mejor fitness
    
    return population[best_index], -results[best_index], routes[best_index]  # Retornar ruta y costo

best_population, best_cost, route = genetic_algorithm(clientes, vehicles, matrix_km, population_size=100, generations=1000, mutation_rate=0.2)

print('\n\nMejor población: ', [i.cliente for i in best_population])
print("Costo de la mejor ruta:", best_cost)
print(f'Viable {check_viability_client([i.cliente for i in best_population])}')
for i,k in route.items():
    print(
        f'\nVehículo = {i} \n Clientes: {k['clientes']}\n Distancia Total: {k['distancia_total']}\n Peso Total: {k['peso_total']}\n Costo Total: {k['costo_total']}'
    )


Generación 0: Mejor costo: 87.25
Generación 1: Mejor costo: 86.26
Generación 2: Mejor costo: 84.58
Generación 3: Mejor costo: 87.55
Generación 4: Mejor costo: 85.46
Generación 5: Mejor costo: 79.84
Generación 6: Mejor costo: 83.06
Generación 7: Mejor costo: 82.39999999999999
Generación 8: Mejor costo: 88.00999999999999
Generación 9: Mejor costo: 82.02
Generación 10: Mejor costo: 81.19
Generación 11: Mejor costo: 77.36000000000001
Generación 12: Mejor costo: 83.72
Generación 13: Mejor costo: 79.55
Generación 14: Mejor costo: 78.87
Generación 15: Mejor costo: 77.28
Generación 16: Mejor costo: 78.46
Generación 17: Mejor costo: 79.57000000000001
Generación 18: Mejor costo: 78.48
Generación 19: Mejor costo: 75.59
Generación 20: Mejor costo: 78.03999999999999
Generación 21: Mejor costo: 78.35
Generación 22: Mejor costo: 77.32
Generación 23: Mejor costo: 78.78999999999999
Generación 24: Mejor costo: 77.88
Generación 25: Mejor costo: 81.02
Generación 26: Mejor costo: 73.89
Generación 27: Mejor

In [18]:
array = ['Cliente_17', 'Cliente_14', 'Cliente_9', 'Cliente_20', 'Cliente_19', 'Cliente_3', 'Cliente_6', 'Cliente_11', 'Cliente_4', 'Cliente_16', 'Cliente_1', 'Cliente_10', 'Cliente_13', 'Cliente_2', 'Cliente_7', 'Cliente_18', 'Cliente_15', 'Cliente_12', 'Cliente_5', 'Cliente_8']
check_viability_client(array)

False

In [24]:
final_routes = []
for i, k in enumerate(route):
    route[k]['vehicle'] = k
    final_routes.append([i,route[k]])
final_routes

[[0,
  {'clientes': ['Cliente_20', 'Cliente_13', 'Cliente_18', 'Cliente_15'],
   'distancia_total': np.float64(59.63),
   'peso_total': 3645.0,
   'costo_total': np.float64(8.35),
   'vehicle': 2}],
 [1,
  {'clientes': ['Cliente_8',
    'Cliente_19',
    'Cliente_3',
    'Cliente_2',
    'Cliente_7'],
   'distancia_total': np.float64(59.69),
   'peso_total': 4508.0,
   'costo_total': np.float64(11.94),
   'vehicle': 3}],
 [2,
  {'clientes': ['Cliente_16', 'Cliente_4', 'Cliente_9'],
   'distancia_total': np.float64(24.09),
   'peso_total': 2747.0,
   'costo_total': np.float64(4.58),
   'vehicle': 4}],
 [3,
  {'clientes': ['Cliente_6', 'Cliente_1'],
   'distancia_total': np.float64(9.31),
   'peso_total': 1730.0,
   'costo_total': np.float64(1.86),
   'vehicle': 1}],
 [4,
  {'clientes': ['Cliente_10', 'Cliente_12', 'Cliente_11'],
   'distancia_total': np.float64(18.68),
   'peso_total': 2710.0,
   'costo_total': np.float64(2.62),
   'vehicle': 6}],
 [5,
  {'clientes': ['Cliente_14', 'Cli